# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.


# Структура данных:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем файлы с данными и изучим общую информацию

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

In [2]:
#Откроем файл
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
#Рассмотрим первые 5 строк таблицы
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
#Просмотрим общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

*Вывод:*
- Таблица содержит 3214 строк и не имеет нулевых значений;
- В таблице 5 столбцов со следующим наименованием: `calls`, `minutes`, `messages`, `mb_used`, `is_ultra`;
- Столбец `is_ultra` в формате `int64`, все остальные столбцы в формате `float64`;
- Столбец `is_ultra` будет взять за целевой признак, остальные признаки помогут в предсказании решений;
- Так как данных немного, то модель может быть переобученой.

## Разделим данные на выборки

**Разобьем выборку по приницу 60/20/20 для оптимального исхода**

In [6]:
train_df, fn_df = train_test_split(df, test_size=0.40, random_state=13032023)
valid_df, test_df = train_test_split(fn_df, test_size=0.50, random_state=13032023)

print('Размер Тренериующей выборки', train_df.shape[0])
print('Размер Валидационной выборки', valid_df.shape[0])
print('Размер Тестовой выборки', test_df.shape[0])

Размер Тренериующей выборки 1928
Размер Валидационной выборки 643
Размер Тестовой выборки 643


**Составим таблицы признаков и целевого признака для каждого Data Frame**

In [7]:
train_df_features = train_df.drop(['is_ultra'], axis=1)
train_df_target = train_df['is_ultra']

valid_df_features = valid_df.drop(['is_ultra'], axis=1)
valid_df_target = valid_df['is_ultra']

test_df_features = test_df.drop(['is_ultra'], axis=1)
test_df_target = test_df['is_ultra']

*Вывод:*
- Выделили признаки из общих данных, это все столбцы за исключением столбца `is_ultra` и назвали их `features`;
- Выделили целевой признак из общих данных - это столбец `is_ultra` и назвали его `target`;
- Далее от общего массива данных отделили 20% тестовых данных;
- Оставшийся массив разделили на тренировочне данные - это 75% данных и валидационные - это 25% данных. В итоге данные разделились в пропорции 60/20/20;
- Обучение будет проводиться на данных `train_df`;
- Валидация последующих моделей будет проведена на данных `valid_df`;
- Наилучшая модель по валидации будет применима на данных `test_df`.

## Исследуем модели

**Модель типа `Дерево решений`**

Изменим в цикле гиперматаметр `max_depth`

In [8]:
best_depth = 0
best_accuracy = 0

for depth in range(1,15):
    model = DecisionTreeClassifier(random_state=13032023, max_depth=depth)
    model.fit(train_df_features, train_df_target)
    valid_predictions = model.predict(valid_df_features)
    accuracy = accuracy_score(valid_df_target, valid_predictions)
    print('Глубина дерева', depth,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy

print('Лучшая глубина дерева', best_depth,'Лучшая точность', best_accuracy) 

Глубина дерева 1 Точность 0.7402799377916018
Глубина дерева 2 Точность 0.7573872472783826
Глубина дерева 3 Точность 0.7698289269051322
Глубина дерева 4 Точность 0.7682737169517885
Глубина дерева 5 Точность 0.7667185069984448
Глубина дерева 6 Точность 0.7682737169517885
Глубина дерева 7 Точность 0.7776049766718507
Глубина дерева 8 Точность 0.76049766718507
Глубина дерева 9 Точность 0.7744945567651633
Глубина дерева 10 Точность 0.7698289269051322
Глубина дерева 11 Точность 0.7620528771384136
Глубина дерева 12 Точность 0.744945567651633
Глубина дерева 13 Точность 0.744945567651633
Глубина дерева 14 Точность 0.7371695178849145
Лучшая глубина дерева 7 Лучшая точность 0.7776049766718507


In [20]:
best_tree_model = DecisionTreeClassifier(random_state=13032023, max_depth=7)
best_tree_model.fit(train_df_features, train_df_target)
valid_predictions = best_tree_model.predict(valid_df_features)
accuracy = accuracy_score(valid_df_target, valid_predictions)
print('Глубина дерева', 7,'Точность',accuracy)

Глубина дерева 7 Точность 0.7776049766718507


**Теперь настроим гиперматаметр `criterion c gini` на `entropy`**

In [21]:
model = DecisionTreeClassifier(random_state=13032023, max_depth=7, criterion="entropy")
model.fit(train_df_features, train_df_target)

valid_predictions = best_tree_model.predict(valid_df_features)

accuracy = accuracy_score(valid_df_target, valid_predictions)

print('Глубина дерева', 7,'Точность',accuracy)

Глубина дерева 7 Точность 0.7776049766718507


*Гиперпараметр `gini` показывает себя лучше*

**Теперь изменим `splitter c best` на `random`**

In [22]:
model = DecisionTreeClassifier(random_state=23042020, max_depth=8, splitter='random')
model.fit(train_df_features, train_df_target)

valid_predictions = best_tree_model.predict(valid_df_features)

accuracy = accuracy_score(valid_df_target, valid_predictions)

print('Глубина дерева', 7,'Точность',accuracy)

Глубина дерева 7 Точность 0.7776049766718507


*Гиперпараметр `best` работает лучше*

**Для модели `дерево решений` написали цикл с перебором глубины от 1 до 14 и вывели список результатов:**
- Для улучшения качества модели установили следующие гиперпараметры: `random_state=13032023` `max_depth=depth` (глубина дерева решений будет изменяться в диапозоне от 1 до 14) `max_features='auto'` - число признаков для выбора расщепления `criterion = 'entropy'` - изменен критерий Джини, который установлен по умолчания, на энтропию;
- По полученным результатам можно сделать вывод, что максимальная доля правильных ответов модели наблюдается при глубине дерева равное 7 (`max_depth = 7`) и составляет 0.7776049766718507.

**Модель типа `Случайный лес`**

*Изменим в цикле гиперпараметр `n_estimators`*

In [54]:
best_estim = 0
best_accuracy = 0

for estim in range(1,20):
    model = RandomForestClassifier(random_state=13032023, n_estimators=estim)
    model.fit(train_df_features, train_df_target)
    valid_predictions = model.predict(valid_df_features)
    accuracy = accuracy_score(valid_df_target, valid_predictions)
    print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Лучшее количество деревьев', best_estim,'Лучшая точность', best_accuracy)

Количество деревьев 1 Точность 0.7262830482115086
Количество деревьев 2 Точность 0.749611197511664
Количество деревьев 3 Точность 0.7527216174183515
Количество деревьев 4 Точность 0.7744945567651633
Количество деревьев 5 Точность 0.76049766718507
Количество деревьев 6 Точность 0.7869362363919129
Количество деревьев 7 Точность 0.7667185069984448
Количество деревьев 8 Точность 0.7776049766718507
Количество деревьев 9 Точность 0.7667185069984448
Количество деревьев 10 Точность 0.7776049766718507
Количество деревьев 11 Точность 0.7651632970451011
Количество деревьев 12 Точность 0.7791601866251944
Количество деревьев 13 Точность 0.7853810264385692
Количество деревьев 14 Точность 0.7838258164852255
Количество деревьев 15 Точность 0.7822706065318819
Количество деревьев 16 Точность 0.7884914463452566
Количество деревьев 17 Точность 0.7884914463452566
Количество деревьев 18 Точность 0.7900466562986003
Количество деревьев 19 Точность 0.7884914463452566
Лучшее количество деревьев 18 Лучшая точнос

In [56]:
best_forest_model = RandomForestClassifier(random_state=13032023, n_estimators=18)
best_forest_model.fit(train_df_features, train_df_target)
valid_predictions = best_forest_model.predict(valid_df_features)
accuracy = accuracy_score(valid_df_target, valid_predictions)
print('Количество деревьев', 18,'Точность',accuracy)

Количество деревьев 18 Точность 0.7900466562986003


**Теперь настроим гиперматаметр `criterion c gini` на `entropy`**

In [58]:
model = RandomForestClassifier(random_state=13032023, n_estimators=18, criterion='entropy')
model.fit(train_df_features, train_df_target)
valid_predictions = model.predict(valid_df_features)
accuracy = accuracy_score(valid_df_target, valid_predictions)
print('Количество деревьев', estim,'Точность',accuracy)

Количество деревьев 19 Точность 0.7884914463452566


*Гиперпараметр `gini` показывает себя лучше*

**Изменим `bootstrap` с `True` на `Flase`**

In [59]:
model = RandomForestClassifier(random_state=13032023, n_estimators=18, bootstrap=False)
model.fit(train_df_features, train_df_target)
valid_predictions = model.predict(valid_df_features)
accuracy = accuracy_score(valid_df_target, valid_predictions)
print('Количество деревьев', estim,'Точность',accuracy)

Количество деревьев 19 Точность 0.7713841368584758


*`bootstrap` с `True` показывает себя лучше*

**Изменим `warm_start` с `False` на `True`**

In [61]:
model = RandomForestClassifier(random_state=13032023, n_estimators=18, warm_start=True)
model.fit(train_df_features, train_df_target)
valid_predictions = model.predict(valid_df_features)
accuracy = accuracy_score(valid_df_target, valid_predictions)
print('Количество деревьев', estim,'Точность',accuracy)

Количество деревьев 19 Точность 0.7900466562986003


*Значения не изменились*

**Для модели `случайный лес` был написан двойной цикл с перебором количества деревьев:**
- Методом подбора максимального количества деревьев и глубины, пришли к выводу, что лучший результат наблюдается при количестве деревьев 18(`n_estimators=18`) и составляет 0.7900466562986003;

Модель типа `Логистическая регрессия`

Изменим гипер параметр `max_iter` в цикле

In [64]:
best_max_iter = 0
best_accuracy = 0

for max_iter in range(1,40):
    model = LogisticRegression(random_state=13032023, max_iter=max_iter)
    model.fit(train_df_features, train_df_target)
    valid_predictions = model.predict(valid_df_features)
    accuracy = accuracy_score(valid_df_target, valid_predictions)
    print('Количество итерраций', max_iter,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_max_iter = max_iter
        best_accuracy = accuracy

print('Лучшая количество итерраций', best_max_iter,'Лучшая точность', best_accuracy)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Количество итерраций 1 Точность 0.6951788491446346
Количество итерраций 2 Точность 0.6951788491446346
Количество итерраций 3 Точность 0.6951788491446346
Количество итерраций 4 Точность 0.6951788491446346
Количество итерраций 5 Точность 0.6951788491446346
Количество итерраций 6 Точность 0.6951788491446346
Количество итерраций 7 Точность 0.6951788491446346
Количество итерраций 8 Точность 0.6951788491446346
Количество итерраций 9 Точность 0.6951788491446346
Количество итерраций 10 Точность 0.6951788491446346
Количество итерраций 11 Точность 0.6951788491446346
Количество итерраций 12 Точность 0.6951788491446346
Количество итерраций 13 Точность 0.6951788491446346
Количество итерраций 14 Точность 0.6951788491446346
Количество итерраций 15 Точность 0.6951788491446346
Количество итерраций 16 Точность 0.6951788491446346
Количество итерраций 17 Точность 0.6951788491446346
Количество итерраций 18 Точность 0.6951788491446346
Количество итерраций 19 Точность 0.6951788491446346
Количество итерраций 

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Количество итерраций 35 Точность 0.7169517884914464
Количество итерраций 36 Точность 0.7169517884914464
Количество итерраций 37 Точность 0.7169517884914464
Количество итерраций 38 Точность 0.7169517884914464
Количество итерраций 39 Точность 0.7169517884914464
Лучшая количество итерраций 27 Лучшая точность 0.7262830482115086


In [71]:
best_reg_model = LogisticRegression(random_state=13032023)
best_reg_model.fit(train_df_features, train_df_target)
valid_predictions = best_reg_model.predict(valid_df_features)
accuracy = accuracy_score(valid_df_target, valid_predictions)
print('Количество итерраций', 30,'Точность',accuracy)


Количество итерраций 30 Точность 0.7169517884914464


Значение по-умолчанию в `max_iter=30` хорошее

Как и у 28, 29 итераций: увеличение в большую сторону ничего не меняет - изменим другой гиперпараметр `solver`

In [78]:
best_solver = ''
best_accuracy = 0

for solve in {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}:
    model = LogisticRegression(random_state=13032023, solver=solve)
    model.fit(train_df_features, train_df_target)
    valid_predictions = model.predict(valid_df_features)
    accuracy = accuracy_score(valid_df_target, valid_predictions)
    print('Solver', solve,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_solver= solve
        best_accuracy = accuracy

print('Лучший solver', best_solver,'Лучшая точность', best_accuracy)

Solver saga Точность 0.6951788491446346
Solver sag Точность 0.6967340590979783
Solver lbfgs Точность 0.7169517884914464
Solver liblinear Точность 0.7371695178849145
Solver newton-cg Точность 0.7402799377916018
Лучший solver newton-cg Лучшая точность 0.7402799377916018


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Лучшим `solver` является `newton-cg`, а не  `lbfgs`, который стоит по умолчанию

Изменим в цикле гиперпараметр С

In [80]:
best_C = 0.0
best_accuracy = 0


for float_c in [x/10 for x in range(1,15)]:
    model = LogisticRegression(random_state=13032023, C=float_c)
    model.fit(train_df_features, train_df_target)
    valid_predictions = model.predict(valid_df_features)
    accuracy = accuracy_score(valid_df_target, valid_predictions)
    print('C=', float_c,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_C = float_c
        best_accuracy = accuracy

print('Лучший C', best_C,'Лучшая точность', best_accuracy) 

C= 0.1 Точность 0.7169517884914464
C= 0.2 Точность 0.7169517884914464
C= 0.3 Точность 0.7169517884914464
C= 0.4 Точность 0.7169517884914464
C= 0.5 Точность 0.7169517884914464
C= 0.6 Точность 0.7169517884914464
C= 0.7 Точность 0.7169517884914464
C= 0.8 Точность 0.7169517884914464
C= 0.9 Точность 0.7169517884914464
C= 1.0 Точность 0.7169517884914464
C= 1.1 Точность 0.7169517884914464
C= 1.2 Точность 0.7169517884914464
C= 1.3 Точность 0.7169517884914464
C= 1.4 Точность 0.7169517884914464
Лучший C 0.1 Лучшая точность 0.7169517884914464


Значение по умолчанию `C=1` показывает лучший результат

*Вывод:*
- Точность модели решающего дерева при глубине 7 равна 0.7776049766718507;
- Точность модели случайного леса при количестве деревьев 18 равна 0.7900466562986003;
- Точность модели логистической регресии равна 0.7169517884914464;


Так как выборка небольшая, то лучший результат показывает решающее древо с гиперпараметром `max_depth=7`

## Проверим модели на тестовой выборке

Оценим точность каждой рассматриваемой модели на тестовой выборке

In [85]:
test_predictions = best_tree_model.predict(test_df_features)
accuracy = accuracy_score(test_df_target, test_predictions)
print('Точность модели решающего дерева на тестовой выборке',accuracy)

test_predictions = best_forest_model.predict(test_df_features)
accuracy = accuracy_score(test_df_target, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

test_predictions = best_reg_model.predict(test_df_features)
accuracy = accuracy_score(test_df_target, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке',accuracy)

Точность модели решающего дерева на тестовой выборке 0.8242612752721618
Точность модели случайного леса на тестовой выборке 0.8164852255054432
Точность модели логистической регресиии на тестовой выборке 0.744945567651633


*Вывод:*

Для решения поставленной задачи можно выбрать модель логичтисеской регсрессии с поправкой, что точность 0.75 достигается округлением до сотых значения 0.744945567651633.

Учитывая разницу между текущей точностью и точностью на валидационной выборке, можно сделать вывод слабой переобученности модели.

## (бонус) Проверка модели на адекватность

In [90]:
test_predictions = best_forest_model.predict(test_df_features)
accuracy = accuracy_score(test_df_target, test_predictions)
precision = precision_score(test_df_target, test_predictions)
recall = recall_score(test_df_target, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

Accuracy = 0.8164852255054432 Precision = 0.7372881355932204 Recall = 0.5


Для оценки адеватности используется `F-мера` - среднее гармоническое между `precision` и `recall`

Если хотя бы один из параметров близок к нулю, то и `F-мера` стремится к 0
Если оба стремятся к 1, то `F-мера` тоже стремится к 1

In [92]:
f_score = f1_score(test_df_target, test_predictions)
print('F-мера =', f_score)

F-мера = 0.5958904109589042


Адекватность модели оценивается по нескольким параметрам. Для модели классификации это `accuracy`, `precision` и `recall`

Качество модели лучше всего отражают `precision` и `recall` и эти метрики складываются в `F-мера` как единую оценку модели.

Модель получилась среднего качества и  для таких задач она адекватна

## Вывод
В итоге были выполнены следующие действия: 

- Разбиты данные на три выборки: обучающая, валидационная и тестовая.
- Исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
- Найдены оптимальные параметры для каждой модели и выбрана одна из них для обучения модели.
- Оценена точность обученной модели.
- Оценена адекватность модели.


Для обучения модели было использована выборка в 2000 значений. Этого может быть недостаточно для повышения ключевых параметров модели классификатора. Увеличение в 10 или 100 увеличит accuracy, precision и recall, а также время обучения.

Адекватность модели повышается с ростом количества значений

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
